# Contrastive Language-Image Pretraining with SogCLR

### **Introduction**

In this tutorial, you will learn how to conduct contrastive language-image pretraining by optimizing the [Global Contrastive Loss](https://arxiv.org/abs/2202.12387) (GCL) on a subset of the [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/) dataset. Also, you will learn how to evaluate the model on retrieval task using the [MSCOCO](https://cocodataset.org/#home) dataset and zero-shot classification task using the [ImageNet](https://www.image-net.org/challenges/LSVRC/index.php) dataset. The code is based on [iSogCLR's](https://github.com/zhqiu/contrastive-learning-iSogCLR) codebase, which includes the implementation of CLIP, SogCLR and iSogCLR.

### Preparation

First, we:

1. Download the source code and data
2. Install required packages

In [ ]:
# 1. Get the code base
!git clone -b project https://github.com/xywei00/csce689_iSogCLR.git iSogCLR

# 2. Set up environment
import os
os.environ["PYTHONPATH"] = os.environ.get("PYTHONPATH", "") + ":./iSogCLR/bimodal_exps"
os.environ['HUGGINGFACE_HUB_CACHE'] = './checkpoints/huggingface'
!mkdir -p checkpoints
!mkdir -p datasets/imagenet

# 3. Mount Drive and Copy Data (The Fix)
# Your specific path from the screenshot
source_folder = "/home/peter/projects/csce-633/data"

if os.path.exists(source_folder):
    print("Folder found. Copying files locally...")
    # We use quotes around the path to handle the spaces in 'school year'
    !cp -v "{source_folder}/clip_train.tar.gz" .
    !cp -v "{source_folder}/cc3m_subset_100k.tar.gz" .
    !cp -v "{source_folder}/mscoco_val.tar.gz" .
    !cp -v "{source_folder}/val.tar" .

    print("Unzipping files...")
    !tar xf clip_train.tar.gz
    !tar xf cc3m_subset_100k.tar.gz -C datasets
    !tar xf mscoco_val.tar.gz -C datasets
    !tar xf val.tar -C datasets/imagenet
else:
    print(f"ERROR: Could not find folder at {source_folder}")
    print("Please verify the folder name in your Drive exactly matches.")

!sed -i 's/safetensors==0.3.3/safetensors/g' ./iSogCLR/requirements_colab.txt

# 4. Install dependencies
!pip install -r ./iSogCLR/requirements_colab.txt

fatal: destination path 'iSogCLR' already exists and is not an empty directory.


Folder found. Copying files locally...
'/home/peter/projects/csce-633/data/clip_train.tar.gz' -> './clip_train.tar.gz'
'/home/peter/projects/csce-633/data/cc3m_subset_100k.tar.gz' -> './cc3m_subset_100k.tar.gz'
^C
'/home/peter/projects/csce-633/data/mscoco_val.tar.gz' -> './mscoco_val.tar.gz'
'/home/peter/projects/csce-633/data/val.tar' -> './val.tar'


### Training

The following command runs the training script to train a ResNet50 (pretrained on ImageNet) and a DistilBERT (pretrained on BookCorpus and English Wikipedia) on the cc3m dataset using the SogCLR loss for 30 epochs with temperature 0.01.

In [1]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new-adamW/train \
    --init_model \
    --use_amp \
    --ita_type isogclr_new \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
Start training
Train Epoch: [0]  [  0/781]  eta: 5:34:59  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 0.2065  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 4.7061  grad_tau_text: 4.2022  b_I: 5.0000  b_T: 3.3342  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 25.7355  data: 1.0072  max mem: 8515
Train Epoch: [0]  [ 50/781]  eta: 0:10:15  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 0.0858  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 4.9870  grad_tau_text: 4.8742  b_I: 5.0000  b_T: 4.2190  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 0.3422  data: 0.0002  max mem: 9212
Train Epoch: [0]  [100/781]  eta: 0:06:45  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 0.0731  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 4

### Evaluation

The following command runs the evaluation script to evaluate the retrieval performance of the trained model on the MSCOCO validation dataset and the zero-shot classification performance on the ImageNet validation dataset. The evaluation command is obtained by appending `--evaluate --checkpoint /path/to/your/checkpoint --zs_dataset imagenet --zs_datafolder /path/to/imagenet/val` to the training command.

In [2]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new-adamW/eval \
    --init_model \
    --use_amp \
    --ita_type isogclr_new \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --evaluate --checkpoint './output/isogclr_new-adamW/train/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
load checkpoint from ./output/isogclr_new-adamW/train/checkpoint_30.pth
Start training
Computing features for evaluation...
Evaluation time 0:00:21
coco val: {'txt_r1': 14.02, 'txt_r5': 33.88, 'txt_r10': 45.94, 'txt_r_mean': 31.28, 'img_r1': 10.872086049022352, 'img_r5': 27.63805030189132, 'img_r10': 38.52213203246831, 'img_r_mean': 25.677422794460664, 'r_mean': 28.478711397230335}
zeroshot: {'zeroshot_top1': 26.442, 'zeroshot_top3': 39.012, 'zeroshot_top5': 44.478, 'zeroshot_top10': 51.774}
Training time 0:02:52


### Benchmarks

The following results are recall at 1 results on the provided MSCOCO and ImageNet datasets. The first row of results are from the model trained using the CLIP loss, and the second row of results are from the model trained using the SogCLR loss. All results are based on a batch size of 128 for 30-epoch pretraining. IR@1 denotes the recall at 1 of image retrieval on MSCOCO, TR@1 denotes the recall at 1 of text retrieval on MSCOCO, and ACC@1 denotes the top 1 accuracy on ImageNet. Average denotes the average of the three metrics.

| Method | MSCOCO TR@1 | MSCOCO IR@1 | ImageNet ACC@1 | Average |
|:----------:|:--------:|:--------:|:--------:|:--------:|
| CLIP | 12.0 | 9.32 | 21.35 | 14.22 |
| SogCLR |  14.38  |  10.73  | 24.54 | 16.55 |